In [65]:
import typing as tp
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [66]:
def func(x: tp.Sequence[float]) -> float:
    return abs(-x[0] + 2 * x[1] + 5) + 0.5 * x[0]**4 + (0.3 * x[0] - 0.2 * x[1])**2

In [84]:
gm = (-0.703641, 2.123456)

# Графічна частина

## Графік самої функції разом із негладкими границями

In [68]:
SIZE = 5
DPI = 201

In [69]:
x1 = np.linspace(-SIZE, SIZE, DPI)
x2 = np.linspace(-SIZE, SIZE, DPI)
X1, X2 = np.meshgrid(x1, x2)
f = func((X1, X2))

In [70]:
# first non-smooth line
l1x1 = np.linspace(-SIZE, SIZE, DPI)
l1x2 = np.array(DPI * [-2])
l1z = func((l1x1, l1x2))

In [71]:
# second non-smooth line
l2x1 = np.linspace(-SIZE, SIZE, DPI)
l2x2 = (l2x1 + 3) / 4
l2z = func((l2x1, l2x2))

In [72]:
fig = go.Figure(data=[
    go.Surface(z=f, x=X1, y=X2),
    go.Scatter3d(
        x=l1x1, y=l1x2, z=l1z,
        marker=dict(color='white', size=1),
        line=dict(color='white', width=5),
        name=''
    ),
    go.Scatter3d(
        x=l2x1, y=l2x2, z=l2z+0.01,
        marker=dict(color='white', size=1),
        line=dict(color='white', width=5),
        name=''
    ),
    go.Scatter3d(
        x=[gm[0]], y=[gm[1]], z=[func(gm)],
        marker=dict(color='white', size=5),
        name=''
    )
])
fig.update_layout(width=800, height=600)
fig.show()

## Графік субградієнту

In [73]:
def sub_grad(x: tp.Sequence[float]) -> tp.Sequence[float]:
    g = [
        2 * x[0]**3 + 1.5
    ]

    if -x[0] + 2 * x[1] + 5 < 0:
        g[0] -= 2
    elif -x[0] + 2 * x[1] + 5 == 0:
        g[0] += np.random.uniform(-2, 2, g[0].shape)
    else:  # -x[0] + 2 * x[1] + 5 > 0
        g[0] += 2

    return np.array(g)

In [74]:
SIZE = 5
DPI = 21

In [75]:
# first non-smooth line
l1x1 = np.linspace(-SIZE, SIZE, DPI)
l1x2 = np.array(DPI * [-2])
l1z = func((l1x1, l1x2))

In [76]:
# second non-smooth line
l2x1 = np.linspace(-SIZE, SIZE, DPI)
l2x2 = (5 + l1x1)/2
l2z = func((l2x1, l2x2))

In [77]:
x1 = np.linspace(-SIZE, SIZE, DPI)
x2 = np.linspace(-SIZE, SIZE, DPI)
X1, X2 = np.meshgrid(x1, x2)
for _ in range(4):
    X1 = np.vstack([X1, l1x1])
    X2 = np.vstack([X2, l1x2])
    X1 = np.vstack([X1, l2x1])
    X2 = np.vstack([X2, l2x2])
f = func((X1, X2))

In [78]:
g = np.array([sub_grad((x1i, x2i)) for (x1i, x2i) in zip(X1.ravel(), X2.ravel())]).reshape((*X1.shape, 2))
u = np.array([np.array([dx for (dx, dy) in gi]) for gi in g])
v = np.array([np.array([dy for (dx, dy) in gi]) for gi in g])

ValueError: cannot reshape array of size 609 into shape (29,21,2)

In [79]:
w = 0.5 * np.sqrt(u**2 + v**2)
u /= w
v /= w

In [85]:
fig = ff.create_quiver(x=X1, y=X2, u=-u, v=-v, name='gradient')
fig.add_trace(
    go.Scatter(x=l2x1, y=l2x2, mode='lines',
               name='$(5 + x)/2= y$',
               line=dict(color='green', width=1),
               connectgaps=True))
fig.add_trace(
    go.Scatter(x=[gm[0]], y=[gm[1]],
               name='optimum'))
fig.update_layout(width=800, height=800)
fig.show()

# Субградієнтний метод

In [17]:
def optimize(
    func: tp.Callable[[tp.Sequence[float]], float],
    sub_grad: tp.Callable[[tp.Sequence[float]], tp.Sequence[float]],
    init: tp.Sequence[float],
    step_rule: str,
    step_params: tp.Mapping[str, float] = {},
    max_iter: int = 10_000,
    tol: float = 1e-6,
) -> tp.Tuple[tp.Sequence[tp.Sequence[float]], tp.Sequence[float]]:
    """Unconditionally optimizes func with subgradient method.

    Parameters:
    func:         R^n -> R function to be optimized.
    sub_grad:     subgradient of func.
    init:         initial value to start the optimization from.
    step_rule:    rule to select the step size.
                  Should be one of:
                    'constant-size'
                    'constant-length'
                    'square-summable'
                    'diminishing'
    step_params:  parameters of the step size rule.
    max_iter:     maximum number of iterations to perform.
    tol:          tolerance up to which the optimization is performed.

    Returns:
    points: a list of visited points.
    values: a list of function values in these points.
    """
    points = [init]
    values = [func(init)]

    for k in range(max_iter):
        g = sub_grad(points[-1])

        step_size: float = 0
        if step_rule == 'constant-size':
            step_size = step_params['h']
        elif step_rule == 'constant-length':
            step_size = step_params['h'] / np.linalg.norm(g)
        elif step_rule == 'square-summable':
            step_size = step_params['a'] / (step_params['b'] + k + 1)
        elif step_rule == 'diminishing':
            step_size = step_params['a'] / np.sqrt(k + 1)
        else:
            raise ValueError("please refer to the docstring for supported values of step_rule")

        points.append(points[-1] - step_size * g)
        values.append(func(points[-1]))

        if np.linalg.norm(points[-1] - points[-2]) < tol and \
           np.linalg.norm(values[-1] - values[-2]) < tol:
            break

    return np.vstack(points), np.vstack(values)

## Порівняння різних розмірів кроку:

In [18]:
COLORS = ['red', 'blue', 'green']
STEPS = [0.1, 0.01, 1e-3]
fig = go.Figure()
for (color, step) in zip(COLORS, STEPS):
    points, values = optimize(func, sub_grad, np.array([0, 0]), 'constant-size', {'h': step}, max_iter=100)
    values = values.ravel()
    fig.add_trace(
        go.Scatter(x=np.arange(len(values)), y=values, mode='lines',
                   name=f'constant-size, {step}',
                   line=dict(color=color, width=1),
                   connectgaps=True)
    )
fig.show()

In [19]:
COLORS = ['red', 'blue', 'green']
STEPS = [0.1, 0.01, 0.001]
fig = go.Figure()
for (color, step) in zip(COLORS, STEPS):
    points, values = optimize(func, sub_grad, np.array([0, 0]), 'constant-length', {'h': step}, max_iter=100)
    values = values.ravel()
    fig.add_trace(
        go.Scatter(x=np.arange(len(values)), y=values, mode='lines',
                   name=f'constant-length, {step}',
                   line=dict(color=color, width=1),
                   connectgaps=True)
    )
fig.show()

Висновки: в обох правилах
- завеликий крок призводить до більших коливань, унаслідок чого алгоритм може не збігатися;
- замалий крок призводить до кращої точності, але вона потребує набагато більше ітерацій.

## Порівняння різних правил вибору кроку

In [20]:
COLORS = ['red', 'blue', 'green']
PARAMS = [{'h': 0.1}, {'a': 1, 'b': 1}, {'a': 0.5}]
METHODS = ['constant-size', 'square-summable', 'diminishing']
fig = go.Figure()
for (color, params, method) in zip(COLORS, PARAMS, METHODS):
    points, values = optimize(func, sub_grad, np.array([0, 0]), method, params, max_iter=100)
    values = values.ravel()
    fig.add_trace(
        go.Scatter(x=np.arange(len(values)), y=values, mode='lines',
                   name=f'{method}, {params}',
                   line=dict(color=color, width=1),
                   connectgaps=True)
    )
fig.show()

Висновки:
- сумовні з квадратом кроки швидко збігаються навіть без підбору параметрів;
- спадні несумовні кроки також збігаються, хоча й не так швидко;
- сталий розмір кроку не обов'язково є поганим, особливо якщо є можливість його підібрати.